# Forecasting Series

https://www.kaggle.com/datasets/veronicataipe/barcelona-atmosphere-data

Sample how a bad choice in data leads to a bad model unable to predict. 

In [1]:
#First steep is import the libraries. 
import tensorflow as tf
tf.random.set_seed(42)
from sklearn.model_selection import train_test_split
#Numpy is a lybrary that allow us to work with arrays. 
import numpy as np
#keras is an open source neural networks lybrary writted in python that run's in varios frameworks, TensorFlow included. 
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import RootMeanSquaredError
import matplotlib.pyplot as plt
import pandas as pd
import csv


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy.stats import linregress

#Tranfer learning from tensorhub
import tensorflow_hub as hub

print (tf.__version__)

gEpochs = 10
WINDOW_SIZE = 5

2.9.0


In [2]:
# Get the data from the df. 
bcntmp = './Data/BCNtemp.csv'
df = pd.read_csv(bcntmp)


In [3]:
# YEAR, MO, DY, HR, PS, T2M
df_work = df[["YEAR", "MO", "DY", "HR", "PS", "T2M"]]
df_work

,YEAR,MO,DY,HR,PS,T2M
0,2001,1,1,0,97.95,3.49
1,2001,1,1,1,97.89,3.65
2,2001,1,1,2,97.82,3.41
3,2001,1,1,3,97.76,3.55
4,2001,1,1,4,97.71,3.92
...,...,...,...,...,...,...
184075,2021,12,31,19,99.14,12.30
184076,2021,12,31,20,99.15,12.08
184077,2021,12,31,21,99.15,11.65
184078,2021,12,31,22,99.14,11.52


In [4]:
df_work.dtypes

YEAR      int64
MO        int64
DY        int64
HR        int64
PS      float64
T2M     float64
dtype: object

In [5]:
#Put the data in a X and y arrays
#We get a number of datas to predict the next one. In case of a window_size of 5
#The shape change to accept multivariate variables, the most important variable comes first. 
#First round x[[t0, ds0, ..., yc0] [t1, ds1, ..., yc1] ...[t4, ds4, ..., yc4]] y[t5][p5]
#First round x[[t1, ds1, ..., yc1] [t2, ds2, ..., yc2] ...[t5, ds5, ..., yc5]] y[t6][p6]
#.... till the end of serie. 
def df_xy_series(df, window_size=WINDOW_SIZE, label1=4, label2=5):
    #Convert dataset to Numpy
    df_as_np = df.to_numpy()
    X=[]
    y=[]
    
    for i in range(len(df_as_np)-window_size):
        #ADDINg AN ENTIRE ROW
        row = [r for r in df_as_np[i:i+window_size]]
        X.append(row)
        #add the 2 labels. 
        label = df_as_np[i+window_size][label1], df_as_np[i+window_size][label2]
        y.append(label)
    
    return np.array(X), np.array(y)

In [6]:
X, y = df_xy_series(df_work)
X.shape, y.shape

((184075, 5, 6), (184075, 2))

In [7]:
X[0]

array([[2.001e+03, 1.000e+00, 1.000e+00, 0.000e+00, 9.795e+01, 3.490e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 1.000e+00, 9.789e+01, 3.650e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 2.000e+00, 9.782e+01, 3.410e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 3.000e+00, 9.776e+01, 3.550e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 4.000e+00, 9.771e+01, 3.920e+00]])

In [8]:
#Split the data 
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)
#X_train.shape, X_val.shape, y_train.shape, y_val.shape 

X_train, y_train = X[:150000], y[:150000]
X_val, y_val = X[150000:], y[150000:]
X_train.shape, X_val.shape, y_train.shape, y_val.shape 

((150000, 5, 6), (34075, 5, 6), (150000, 2), (34075, 2))

In [9]:
X_train[0]

array([[2.001e+03, 1.000e+00, 1.000e+00, 0.000e+00, 9.795e+01, 3.490e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 1.000e+00, 9.789e+01, 3.650e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 2.000e+00, 9.782e+01, 3.410e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 3.000e+00, 9.776e+01, 3.550e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 4.000e+00, 9.771e+01, 3.920e+00]])

In [10]:
#Standarize the input training data
def preprocess_X(X, cols=[0]):
    for n in cols: 
        mean = np.mean(X[:, :, n])
        std = np.std(X[:, :, n])
        X[:, :, n] = (X[:, :, n] - mean) / std
        print ("trreating col X {n}",  n)
    return X
        
def preprocess_y(y, cols=[0]):
    for n in cols: 
        mean = np.mean(y[ :, n])
        std = np.std(y[:, n])
        y[:, n] = (y[:, n] - mean) / std
        print ("treating col y {n}",  n)
    return y

In [11]:

#X_train = preprocess_X (X_train, cols=[0, 1, 2, 3, 5])


In [12]:
X_train[0]

array([[2.001e+03, 1.000e+00, 1.000e+00, 0.000e+00, 9.795e+01, 3.490e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 1.000e+00, 9.789e+01, 3.650e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 2.000e+00, 9.782e+01, 3.410e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 3.000e+00, 9.776e+01, 3.550e+00],
       [2.001e+03, 1.000e+00, 1.000e+00, 4.000e+00, 9.771e+01, 3.920e+00]])

In [13]:
y_train[0]

array([97.69,  4.28])

In [14]:
#y_train = preprocess_y(y_train, [0, 1])

In [15]:
y_train[0]

array([97.69,  4.28])

In [16]:
# Define different models
def get_model(kindmodel, varsInput = 1, window_size = WINDOW_SIZE):
    switcher = {
        #LSTM Bidirectionsl
        12:tf.keras.models.Sequential([
            tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                     strides=1,
                     activation="relu",
                     padding='causal',
                     input_shape=[window_size, varsInput]),
            tf.keras.layers.LSTM(64, return_sequences=True),
            tf.keras.layers.LSTM(64),
            tf.keras.layers.Dense(8, activation="relu"),
            tf.keras.layers.Dense(4, activation="relu"),
            tf.keras.layers.Dense(2)]),
        #convolutiona + Maxpooling
        2:tf.keras.models.Sequential([
            tf.keras.layers.Conv1D(filters=64, kernel_size=2,
                     activation="relu",
                     input_shape=[window_size, varsInput]), 
            tf.keras.layers.MaxPooling1D(pool_size=2), 
            tf.keras.layers.Flatten(), 
            tf.keras.layers.Dense(8, activation='relu'), 
            tf.keras.layers.Dense(2)]), 
        #Input Layer LSTM
        3:tf.keras.models.Sequential([
            tf.keras.layers.InputLayer((window_size, varsInput)), 
            tf.keras.layers.LSTM(64), 
            tf.keras.layers.Dense(8, activation='relu'), 
            tf.keras.layers.Dense(2, activation="linear")]), 
        4:tf.keras.models.Sequential([
            tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                     strides=1,
                     activation="relu",
                     padding='causal',
                     input_shape=[window_size, varsInput]),
            tf.keras.layers.Flatten(), 
            tf.keras.layers.Dense(8, activation="relu"),
            tf.keras.layers.Dense(2)]),
    }
    return switcher.get(kindmodel, None)

In [17]:
model12 = get_model(3, varsInput = 6)
model12.summary()
cp12 = ModelCheckpoint('model12/', save_best_only=True)
try:
    model12.predict(X_train)
except:
    print("Architecture and dataset incompatible")
else:
    print("compatible! Congrats!")

Metal device set to: Apple M1 Pro


2022-08-08 00:05:02.966435: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-08 00:05:02.966542: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 64)                18176     
                                                                 
 dense_5 (Dense)             (None, 8)                 520       
                                                                 
 dense_6 (Dense)             (None, 2)                 18        
                                                                 
Total params: 18,714
Trainable params: 18,714
Non-trainable params: 0
_________________________________________________________________


2022-08-08 00:05:03.501647: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  44/4688 [..............................] - ETA: 11s 

2022-08-08 00:05:03.726069: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-08 00:05:03.762398: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4688/4688 [==============================] - 11s 2ms/step
compatible! Congrats!


In [18]:
model12.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer="Adam", metrics=[RootMeanSquaredError()])
model12.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=gEpochs, callbacks=[cp12])

Epoch 1/10


2022-08-08 00:05:16.214470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-08 00:05:16.328201: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-08 00:05:16.410223: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4688/4688 [==============================] - ETA: 0s - loss: 709.0131 - root_mean_squared_error: 26.6273

2022-08-08 00:05:52.752151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-08 00:05:52.793214: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: model12/assets


...

INFO:tensorflow:Assets written to: model12/assets


4688/4688 [==============================] - 41s 9ms/step - loss: 29.0180 - root_mean_squared_error: 5.3868 - val_loss: 26.3831 - val_root_mean_squared_error: 5.1365
Epoch 4/10
4688/4688 [==============================] - 37s 8ms/step - loss: 29.0175 - root_mean_squared_error: 5.3868 - val_loss: 26.4501 - val_root_mean_squared_error: 5.1430
Epoch 5/10
4688/4688 [==============================] - 38s 8ms/step - loss: 29.0226 - root_mean_squared_error: 5.3873 - val_loss: 26.4332 - val_root_mean_squared_error: 5.1413
Epoch 6/10
4688/4688 [==============================] - 38s 8ms/step - loss: 29.0219 - root_mean_squared_error: 5.3872 - val_loss: 26.7179 - val_root_mean_squared_error: 5.1689
Epoch 7/10
4688/4688 [==============================] - 38s 8ms/step - loss: 29.0192 - root_mean_squared_error: 5.3870 - val_loss: 26.4867 - val_root_mean_squared_error: 5.1465
Epoch 8/10
4688/4688 [==============================] - 38s 8ms/step - loss: 29.0185 - root_mean_squared_error: 5.3869 - val_l

INFO:tensorflow:Assets written to: model12/assets


INFO:tensorflow:Assets written to: model12/assets


4688/4688 [==============================] - 41s 9ms/step - loss: 29.0150 - root_mean_squared_error: 5.3866 - val_loss: 26.3819 - val_root_mean_squared_error: 5.1363
Epoch 10/10
4688/4688 [==============================] - 37s 8ms/step - loss: 29.0218 - root_mean_squared_error: 5.3872 - val_loss: 26.4362 - val_root_mean_squared_error: 5.1416


In [21]:
def get_predictions(model, X, y):
    predictions=model.predict(X)
    
    p_preds, temp_preds = predictions[:, 0], predictions[:, 1]
    p_actuals, temp_actuals = y[:, 0], y[:, 1]
    results = pd.DataFrame(data={'Temperature Predictions': temp_preds,
                            'Temperature Actuals':temp_actuals,
                            'Pressure Predictions': p_preds,
                            'Pressure Actuals': p_actuals
                            })
    return predictions, results


In [22]:
train_predictions, train_results= get_predictions(
    model12, X_val, y_val)
train_results

1065/1065 [==============================] - 3s 2ms/step


,Temperature Predictions,Temperature Actuals,Pressure Predictions,Pressure Actuals
0,15.634737,3.05,98.17482,98.09
1,15.634737,2.94,98.17482,98.10
2,15.634737,3.63,98.17482,98.11
3,15.634737,5.87,98.17482,98.10
4,15.634737,7.76,98.17482,98.09
...,...,...,...,...
34070,15.634737,12.30,98.17482,99.14
34071,15.634737,12.08,98.17482,99.15
34072,15.634737,11.65,98.17482,99.15
34073,15.634737,11.52,98.17482,99.14
